In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import os 
from os import listdir
from random import sample
import pandas as pd
import random
np.random.seed(2)
np.random.RandomState(2)
import sklearn

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Set up path to image dataset, create list of file names.

In [ ]:
data_path = '/content/drive/MyDrive/Capstone/raw_dataset'

In [ ]:
data_files = os.listdir(data_path)

## Write Image Folder to CSV

The image name and path are now written to a CSV that is saved in the file.

In [ ]:
#Code guidance received from:
#https://gist.github.com/GermanCM/03754e11ac7e9a6343754ff389eb47f0
#https://stackoverflow.com/questions/58625312/printing-file-name-and-file-path-to-a-csv

with open('/content/drive/MyDrive/Capstone/dataset.csv', 'w') as fout:
    fout.write('file_name,file_path\n')
    for root, dirs, files in os.walk(data_path):
        for x in files:
            file_path= os.path.join(root, x)
            file_name= os.path.basename(x)
            fout.write('{},{}\n'.format(file_name,file_path))

In [ ]:
clothes_df = pd.read_csv('/content/drive/MyDrive/Capstone/dataset.csv')

In [ ]:
clothes_df.head()

,file_name,file_path
0,vertical_top_1012.jpg,/content/drive/MyDrive/Capstone/raw_dataset/ve...
1,horiz_tshirt_1020.jpg,/content/drive/MyDrive/Capstone/raw_dataset/ho...
2,horiz_sweater_1001.jpg,/content/drive/MyDrive/Capstone/raw_dataset/ho...
3,horiz_top_1000.jpg,/content/drive/MyDrive/Capstone/raw_dataset/ho...
4,horiz_top_1003.jpg,/content/drive/MyDrive/Capstone/raw_dataset/ho...


The naming convention of the files, when saved into the raw dataset, included the assigned pattern label, an underscore, and the assigned garment label.

In [ ]:
clothes_df['pattern_label'] = [i.split('_')[0] for i in clothes_df['file_name']]
clothes_df['garment_label'] = [i.split('_')[1] for i in clothes_df['file_name']]

Fixing typo in naming convention.

In [ ]:
clothes_df['pattern_label'] = clothes_df['pattern_label'].replace('Copy of animal','animal')

There is an imbalance in the dataset regarding pattern labeling, with solid patterned clothing comprising 56.9% of the images.

In [ ]:
clothes_df['pattern_label'].value_counts(normalize=True)

solid        0.567527
horiz        0.161798
polka        0.091426
vertical     0.040971
checkered    0.039264
chevron      0.039264
animal       0.036039
paisley      0.023710
Name: pattern_label, dtype: float64

In [ ]:
clothes_df['garment_label'].value_counts(normalize=True)

dress      0.139226
sweater    0.136002
coat       0.134484
pants      0.129932
tshirt     0.116654
top        0.107360
jacket     0.087253
shorts     0.086495
skirt      0.062595
Name: garment_label, dtype: float64

## Resampling for Pattern Modeling

In [ ]:
solids = clothes_df[clothes_df["pattern_label"] == "solid"]

Only 50% of the solid patterned images will be included in the resampled dataset. The number of images featuring polkadot, vertical striped, chevron striped, checkered, animal print, or paisley patterned clothing will be doubled.

In [ ]:
solid_downsample = sklearn.utils.resample(solids,
             replace=True,
             n_samples=int(len(solids)/5),
             random_state=2)

non_solid = clothes_df[clothes_df["pattern_label"] != "solid"]


data_resampled = pd.concat([solid_downsample, non_solid])

The value count breakdown of the resampled dataset are below. While there remains an imbalance in the dataset, it has been reduced.

In [ ]:
data_resampled['pattern_label'].value_counts(normalize=True)

horiz        0.296386
solid        0.207783
polka        0.167477
vertical     0.075052
chevron      0.071925
checkered    0.071925
animal       0.066018
paisley      0.043433
Name: pattern_label, dtype: float64

In [ ]:
clothes_df['garment_label'].value_counts(normalize=True)

dress      0.139226
sweater    0.136002
coat       0.134484
pants      0.129932
tshirt     0.116654
top        0.107360
jacket     0.087253
shorts     0.086495
skirt      0.062595
Name: garment_label, dtype: float64

## The datasets will now be shuffled and saved down.

In [ ]:
clothes_df = clothes_df.sample(frac=1,random_state=2)
clothes_df.to_csv('/content/drive/MyDrive/Capstone/dataset.csv',index=False)

In [ ]:
data_resampled = data_resampled.sample(frac=1,random_state=2)
data_resampled.to_csv('/content/drive/MyDrive/Capstone/data_resampled.csv',index=False)